In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-24 02:21:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-11-24 02:21:31 (10.9 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
# Extract the dataset from Deliverable 1
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Home_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [7]:
# Re-create the vine_table dataframe
# vine_df = df.select([])

# Datatypes from SQL from pgAdmin
#CREATE TABLE vine_table (
#  review_id TEXT PRIMARY KEY,
#  star_rating INTEGER,
#  helpful_votes INTEGER,
#  total_votes INTEGER,
#  vine TEXT,
#  verified_purchase TEXT

vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1UUISQ1GKOJTI|          1|            0|          0|   N|                Y|
|R1HOJ9WE8VCVOD|          5|            9|          9|   N|                Y|
| RDNGVXMWQN2TN|          5|            0|          0|   N|                Y|
|R3OM9COQMVTDJ2|          2|            0|          0|   N|                Y|
|R3OFUQVR4Y80Q9|          4|            0|          0|   N|                N|
|R2GD25SBBGRKPO|          5|            0|          0|   N|                Y|
|R3RRSLYKVWH9WB|          5|            0|          0|   N|                Y|
|R2HMFAGJJU6NT3|          5|            0|          0|   N|                Y|
|R2QDRZATHBY4GY|          1|            0|          0|   N|                Y|
|R107946YZK57Q2|          2|            0|          0|   N|     

In [9]:
# Filter data & create new dataframe where total_votes >= 20
# df.filter((df.d<5)& (df.col2!=df.col4) & (df.col1==df.col3)).show()

vine_over20votes_df = vine_df.filter((vine_df.total_votes>=20))
vine_over20votes_df.sort('total_votes').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RC0MC0797AI8K|          4|           20|         20|   N|                Y|
|R15PS4LQA8LYWA|          1|           17|         20|   N|                Y|
| RHBNOJACKQ0CW|          5|           20|         20|   N|                Y|
|R1OI8MK5A9ALPW|          1|           14|         20|   N|                Y|
| RHK4VGSHBVOT4|          5|           19|         20|   N|                N|
|R1E71VRDB4EMQX|          2|           16|         20|   N|                Y|
|R2RHKSK8O6KDNS|          5|           20|         20|   N|                Y|
|R2TS8H10UI4FTO|          2|           20|         20|   N|                Y|
|R1F0JQTAJWKFZ1|          1|            2|         20|   N|                Y|
|R2TMA4BRGBYGYI|          4|           18|         20|   N|     

In [10]:
# Filter vine_over20votes_df where # helpful_votes / total_votes >= 50%

vine_helpfulvotes_df = vine_vote_over20_df.filter( ( vine_over20votes_df.helpful_votes / vine_over20votes_df.total_votes ) >= 0.5 )
vine_helpfulvotes_df.sort('total_votes').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RHBNOJACKQ0CW|          5|           20|         20|   N|                Y|
|R14UUFAZJJJCGZ|          1|           14|         20|   N|                N|
| RHK4VGSHBVOT4|          5|           19|         20|   N|                N|
| RC0MC0797AI8K|          4|           20|         20|   N|                Y|
|R2RHKSK8O6KDNS|          5|           20|         20|   N|                Y|
|R1OI8MK5A9ALPW|          1|           14|         20|   N|                Y|
| RK0A5WGZOWCD0|          1|           20|         20|   N|                Y|
|R193XSALMROWTB|          5|           19|         20|   N|                Y|
|R3STEAIY9N5UYK|          4|           18|         20|   N|                Y|
|R2TS8H10UI4FTO|          2|           20|         20|   N|     

In [12]:
# Filter vine_helpfulvotes_df to only include instances where review was written as part of Vine program (paid) (vine == 'Y')
vine_paid_df = vine_helpfulvotes_df.filter(  vine_helpfulvotes_df.vine == 'Y' )
vine_paid_df.sort('total_votes').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2GDI32JVTNQM5|          1|           16|         20|   Y|                N|
|R2S1WGA8JCN9UT|          4|           15|         20|   Y|                N|
| RRKJH61ECNQ7P|          5|           16|         20|   Y|                N|
|R3H90SV50CNOTV|          5|           17|         20|   Y|                N|
|R3TJXGBHPZ10IQ|          4|           20|         20|   Y|                N|
|R297B2740YOVXY|          5|           20|         20|   Y|                N|
|R3NJTPBICFJCIU|          1|           19|         20|   Y|                N|
|R15AGK9M9WTFVY|          4|           15|         20|   Y|                N|
|R3PHHUAFTNHJOZ|          5|           18|         20|   Y|                N|
| RNWGFOFNAN8F2|          5|           20|         20|   Y|     

In [13]:
# Filter vine_helpfulvotes_df to only include instances where review was NOT written as part of Vine program (unpaid) (vine == 'N')
vine_unpaid_df = vine_helpfulvotes_df.filter(  vine_helpfulvotes_df.vine == 'N' )
vine_unpaid_df.sort('total_votes').show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RC0MC0797AI8K|          4|           20|         20|   N|                Y|
|R1OCFMKI32AJ5G|          5|           16|         20|   N|                Y|
| RHBNOJACKQ0CW|          5|           20|         20|   N|                Y|
|R1OI8MK5A9ALPW|          1|           14|         20|   N|                Y|
| RHK4VGSHBVOT4|          5|           19|         20|   N|                N|
|R193XSALMROWTB|          5|           19|         20|   N|                Y|
|R2RHKSK8O6KDNS|          5|           20|         20|   N|                Y|
|R3GU53VLHGWU0W|          5|           20|         20|   N|                Y|
| RK0A5WGZOWCD0|          1|           20|         20|   N|                Y|
|R3QGCAXCQB9CYD|          5|           18|         20|   N|     

In [18]:
#For both paid and unpaid reviews, determine:
#1) Total number of reviews
vine_paid_Total = vine_paid_df.count()
vine_unpaid_Total = vine_unpaid_df.count()

#2) Number of 5-star reviews
vine_paid_5star_Total = vine_paid_df.filter(vine_paid_df.star_rating == '5').count()
vine_unpaid_5star_Total = vine_unpaid_df.filter(vine_unpaid_df.star_rating == '5').count()

#3) Percent of 5-star reviews
vine_paid_5star_percent = ( vine_paid_5star_Total / vine_paid_Total ) * 100
vine_unpaid_5star_percent = ( vine_unpaid_5star_Total / vine_unpaid_Total ) * 100

In [20]:
print('Vine Paid Reviews')
print('Total 5-Star Reviews for Vine Paid Subscribers is ' + str(vine_paid_5star_Total))
print('Total Reviews for Vine Paid Subscribers is ' + str(vine_paid_Total))
print('Percentage of 5-Star Paid Reviews: ' + str(round(vine_paid_5star_percent,2)) + '%')
print('****************************************************************')
print('Vine Un-Paid Reviews')
print('Total 5-Star Reviews is ' + str(vine_unpaid_5star_Total))
print('Total Reviews is ' + str(vine_unpaid_Total))
print('Percentage of 5-Star Unpaid Reviews: ' + str(round(vine_unpaid_5star_percent,2)) + '%')

Vine Paid Reviews
Total 5-Star Reviews for Vine Paid Subscribers is 647
Total Reviews for Vine Paid Subscribers is 1448
Percentage of 5-Star Paid Reviews: 44.68%
****************************************************************
Vine Un-Paid Reviews
Total 5-Star Reviews is 44104
Total Reviews is 90768
Percentage of 5-Star Unpaid Reviews: 48.59%
